In [0]:
from pyspark.sql.functions import explode, sequence, to_date

In [0]:
spark.sparkContext.getConf().getAll()

Out[36]: [('spark.databricks.preemption.enabled', 'true'),
 ('spark.databricks.clusterUsageTags.clusterFirstOnDemand', '1'),
 ('spark.sql.hive.metastore.jars', '/databricks/databricks-hive/*'),
 ('spark.driver.tempDirectory', '/local_disk0/tmp'),
 ('spark.sql.warehouse.dir', 'dbfs:/user/hive/warehouse'),
 ('spark.databricks.managedCatalog.clientClassName',
 'com.databricks.managedcatalog.ManagedCatalogClientImpl'),
 ('spark.app.id', 'app-20220902094120-0000'),
 ('spark.hadoop.fs.gs.impl',
 'shaded.databricks.com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem'),
 ('spark.executor.extraJavaOptions',
 '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED -Djava.io.tmpdir=/local_disk0/tmp -XX:ReservedCodeCacheSize=512m -XX:+UseCodeCacheFlushing -Djava.security.properties=/databricks/spark/dbconf/java/extra.security -XX:-UseContainerSupport -XX:+PrintFlagsFinal -XX:+PrintGCDateStamps -XX:+PrintGCDetails -verbose:gc -Xss4m -Djava.library.path=/usr/java/packages/lib/amd64:/usr/lib64:/lib64:/lib:/usr/lib:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni -Djavax.xml.datatype.DatatypeFactory=com.sun.org.apache.xerces.internal.jaxp.datatype.DatatypeFactoryImpl -Djavax.xml.parsers.DocumentBuilderFactory=com.sun.org.apache.xerces.internal.jaxp.DocumentBuilderFactoryImpl -Djavax.xml.parsers.SAXParserFactory=com.sun.org.apache.xerces.internal.jaxp.SAXParserFactoryImpl -Djavax.xml.validation.SchemaFactory:http://www.w3.org/2001/XMLSchema=com.sun.org.apache.xerces.internal.jaxp.validation.XMLSchemaFactory -Dorg.xml.sax.driver=com.sun.org.apache.xerces.internal.parsers.SAXParser -Dorg.w3c.dom.DOMImplementationSourceList=com.sun.org.apache.xerces.internal.dom.DOMXSImplementationSourceImpl -Djavax.net.ssl.sessionCacheSize=10000 -Dscala.reflect.runtime.disable.typetag.cache=true -Dcom.google.cloud.spark.bigquery.repackaged.io.netty.tryReflectionSetAccessible=true -Dlog4j2.formatMsgNoLookups=true -Ddatabricks.serviceName=spark-executor-1'),
 ('spark.executor.memory', '7284m'),
 ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '1'),
 ('spark.hadoop.fs.fcfs-s3.impl.disable.cache', 'true'),
 ('spark.hadoop.fs.s3a.retry.limit', '20'),
 ('spark.sql.streaming.checkpointFileManagerClass',
 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'),
 ('spark.databricks.clusterUsageTags.clusterLastActivityTime',
 '1661829498609'),
 ('spark.databricks.service.dbutils.repl.backend',
 'com.databricks.dbconnect.ReplDBUtils'),
 ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'),
 ('spark.databricks.clusterSource', 'UI'),
 ('spark.databricks.clusterUsageTags.sparkVersion', '10.4.x-scala2.12'),
 ('spark.hadoop.hive.server2.transport.mode', 'http'),
 ('spark.databricks.acl.dfAclsEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.effectiveSparkVersion',
 '10.4.x-scala2.12'),
 ('spark.hadoop.fs.cpfs-adl.impl.disable.cache', 'true'),
 ('spark.hadoop.spark.databricks.io.parquet.verifyChecksumOnWrite.throwsException',
 'true'),
 ('spark.databricks.clusterUsageTags.hailEnabled', 'false'),
 ('spark.hadoop.fs.mcfs-s3.impl',
 'com.databricks.sql.acl.fs.ManagedCatalogFileSystem'),
 ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.containerType', 'LXC'),
 ('spark.eventLog.enabled', 'false'),
 ('spark.driver.extraJavaOptions',
 '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/ja

### Read the uploaded files into DataFrames

In [0]:
payments_df = spark.read.format("csv") \
      .option("sep", ",") \
      .load("/FileStore/data/payments.csv") 

trips_df = spark.read.format("csv") \
      .option("sep", ",") \
      .load("/FileStore/data/trips.csv") 

riders_df = spark.read.format("csv") \
      .option("sep", ",") \
      .load("/FileStore/data/riders.csv") 

stations_df = spark.read.format("csv")\
    .option("sep",",")\
    .load("/FileStore/data/stations.csv")

### Renaming Columns and displaying the data

In [0]:
trips_list = ["trip_id" ,"rideable_type", "start_at", "ended_at", "start_station_id", "end_station_id", "rider_id"] 
trips_df = trips_df.toDF(*trips_list)
trips_df.show()

payments_list = ["payment_id","date","amount","rider_id" ]
payments_df = payments_df.toDF(*payments_list)
payments_df.show()

stations_list = ["station_id","name","latitude","longitude" ]
stations_df = stations_df.toDF(*stations_list)
stations_df.show()

riders_list = ["rider_id", "first_name", "last_name", "address", "birthday", "account_start_date", "account_end_date", "is_member" ]
riders_df = riders_df.toDF(*riders_list)
riders_df.show()

+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
 trip_id|rideable_type| start_at| ended_at|start_station_id|end_station_id|rider_id|
+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43| 525| 660| 71934|
0FEFDE2603568365| classic_bike|2021-02-14 17:52:38|2021-02-14 18:12:09| 525| 16806| 47854|
E6159D746B2DBB91|electric_bike|2021-02-09 19:10:18|2021-02-09 19:19:10| KA1503000012| TA1305000029| 70870|
B32D3199F1C2E75B| classic_bike|2021-02-02 17:49:41|2021-02-02 17:54:06| 637| TA1305000034| 58974|
83E463F23575F4BF|electric_bike|2021-02-23 15:07:23|2021-02-23 15:22:37| 13216| TA1309000055| 39608|
BDAA7E3494E8D545|electric_bike|2021-02-24 15:43:33|2021-02-24 15:49:05| 18003| KP1705001026| 36267|
A772742351171257| classic_bike|2021-02-01 17:47:42|2021-02-01 17:48:33| KP1705001026| KP1705001026| 50104|
295476889D9B79F8| classic_bike|2021-02-11 18:33:53|2021-02-11 18:35:09| 18003| 18003| 19618|
362087194BA4CC9A| classic_bike|2021-02-27 15:13:39|2021-02-27 15:36:36| KP1705001026| KP1705001026| 16732|
21630F715038CCB0| classic_bike|2021-02-20 08:59:42|2021-02-20 09:17:04| KP1705001026| KP1705001026| 57068|
A977EB7FE7F5CD3A| classic_bike|2021-02-20 08:58:16|2021-02-20 08:58:41| KP1705001026| KP1705001026| 32712|
8B868B03D6753C2A| classic_bike|2021-02-20 16:45:11|2021-02-20 16:59:47| KP1705001026| KP1705001026| 23227|
BD331D658B9D2C31| classic_bike|2021-02-18 13:21:03|2021-02-18 13:25:20| 525| 520| 73221|
8DFEA9BAFE6BAA62| classic_bike|2021-02-26 17:40:05|2021-02-26 17:42:49| 13253| TA1309000050| 22163|
27BE9F6E67AFD86C| classic_bike|2021-02-06 14:40:25|2021-02-06 14:55:50| 525| 15578| 7566|
9B790D47A0A0F7F1| classic_bike|2021-02-19 23:25:40|2021-02-20 00:10:00| KA1503000044| KA1504000142| 71588|
3C2DF72600B1DE6C| classic_bike|2021-02-18 23:20:10|2021-02-19 00:01:39| KA1503000044| KA1504000142| 38661|
48A8D07ED9C7065C| classic_bike|2021-02-20 23:35:29|2021-02-21 00:17:18| KA1503000044| KA1504000142| 64751|
BBFF2AAA0A3A1A26|electric_bike|2021-02-02 15:48:52|2021-02-02 16:03:40| KA1504000140| 17660| 10721|
030723CBA8CF05E7| classic_bike|2021-02-23 07:44:12|2021-02-23 07:48:57| TA1305000032| 15542| 13281|
+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
only showing top 20 rows

+----------+----------+------+--------+
payment_id| date|amount|rider_id|
+----------+----------+------+--------+
 1|2019-05-01| 9.0| 1000|
 2|2019-06-01| 9.0| 1000|
 3|2019-07-01| 9.0| 1000|
 4|2019-08-01| 9.0| 1000|
 5|2019-09-01| 9.0| 1000|
 6|2019-10-01| 9.0| 1000|
 7|2019-11-01| 9.0| 1000|
 8|2019-12-01| 9.0| 1000|
 9|2020-01-01| 9.0| 1000|
 10|2020-02-01| 9.0| 1000|
 11|2020-03-01| 9.0| 1000|
 12|2020-04-01| 9.0| 1000|
 13|2020-05-01| 9.0| 1000|
 14|2020-06-01| 9.0| 1000|
 15|2020-07-01| 9.0| 1000|
 16|2020-08-01| 9.0| 1000|
 17|2020-09-01| 9.0| 1000|
 18|2020-10-01| 9.0| 1000|
 19|2020-11-01| 9.0| 1000|
 20|2020-12-01| 9.0| 1000|
+----------+----------+------+--------+
only showing top 20 rows

+------------+--------------------+------------------+------------------+
 station_id| name| latitude| longitude|
+------------+--------------------+------------------+------------------+
 525|Glenwood Ave & To...| 42.012701|-87.66605799999999|
KA1503000012| Clark St & Lake St| 41.88579466666667|-87.63110066666668|
 637|Wood St & Chicago...| 41.895634| -87.672069|
 13216| State St & 33rd St| 41.8347335| -87.6258275|
 18003|Fairbanks St & Su...| 41.89580766666667|-87.62025316666669|
KP1705001026|LaSalle Dr & Huro...| 41.894877| -87.632326|
 13253|Lincoln Ave & Wav...| 41.948797| -87.675278|
KA1503000044|Rush St & Hubbard St| 41.890173|-87.62618499999999|
KA1504000140|Winchester Ave & ...| 41.92403733333333|-87.67641483333334|
TA1305000032|Clinton St & Madi...| 41.882242|-87.64106600000001|
TA1306000012| Wells St 

### Creation of Bronze Layer. These tables hold the data in the raw format (same as the files)

In [0]:
#Store into delta files
payments_df.write.format("delta").mode("overwrite").save("payments_data")
trips_df.write.format("delta").mode("overwrite").save("trips_data")
riders_df.write.format("delta").mode("overwrite").save("riders_data")
stations_df.write.format("delta").mode("overwrite").save("stations_data")

#create bronze layer tables
spark.sql("CREATE TABLE bronze_payments USING DELTA LOCATION '/delta/payments_data'")
spark.sql("CREATE TABLE bronze_trips USING DELTA LOCATION '/delta/trips_data'")
spark.sql("CREATE TABLE bronze_riders USING DELTA LOCATION '/delta/riders_data'")
spark.sql("CREATE TABLE bronze_stations USING DELTA LOCATION '/delta/stations_data'")

## Creation of Silver Layer.

### Creating the Date Dimension Table

In [0]:
dates = spark.sql("SELECT to_timestamp(max(ended_at)),to_timestamp(date_add(min(start_at), -1500)) FROM bronze_trips")
beginDate = dates.first()[1]
endDate = dates.first()[0]

(
  spark.sql(f"select explode(sequence(to_timestamp('{beginDate}'), to_timestamp('{endDate}'), interval 1 day)) as calendarDate")
    .createOrReplaceTempView('dimdates')
)

dimDate_df = spark.sql('''SELECT calendarDate as date_key,
                    calendarDate as datetime,
                    date_part('YEAR',calendarDate)  as Year,
                    date_part('QUARTER', calendarDate) as Quarter,
                    date_part('MONTH', calendarDate) as Month,
                    date_part('DAY',calendarDate) as Day,
                    date_part('WEEK',calendarDate) as Week,
                    date_part('HOUR',calendarDate) as Hour,
                    date_part('MINUTE',calendarDate) as Minute,
                    date_part('SECOND',calendarDate) as Second,
                    CASE WHEN date_part('DAYOFWEEK',calendarDate) IN (6,7) THEN 1 ELSE 0 END as is_weekend                    
                    FROM dimDates
                    ''')
dimDate_df.write.format("delta").mode("overwrite").saveAsTable("silver_dimDate")

### Creating the Rider Dimension Table

In [0]:
dimRider_df = spark.sql('''SELECT DISTINCT rider_id as rider_key,
                            rider_id as rider_id,
                            first_name as first_name,
                            last_name as last_name,
                            to_timestamp(birthday) as birthday,
                            is_member as ismember
                            FROM bronze_riders
                            ''')
dimRider_df.write.format("delta").mode("overwrite").saveAsTable("silver_dimRider")

### Creating the Station Dimension Table

In [0]:
dimStation_df = spark.sql('''SELECT DISTINCT station_id as station_key,
                            station_id as station_id,
                            name as name,
                            latitude as latitude,
                            longitude as longitude
                            FROM bronze_stations''');

dimStation_df.write.format("delta").mode("overwrite").saveAsTable("silver_dimStation")

### Creating the Payments Fact Table

In [0]:
factPayment_df = spark.sql('''SELECT DISTINCT p.payment_id as payment_key,
                                p.payment_id as payment_id,
                                p.amount as amount,
                                d.date_key as date_key,
                                r.rider_key as rider_key
                                FROM bronze_payments p
                                JOIN silver_dimDate d ON p.date = to_date(d.date_key)
                                JOIN silver_dimRider r on p.rider_id = r.rider_key''')
factPayment_df.write.format("delta").mode("overwrite").saveAsTable("silver_factPayment")

### Creating the Trips Fact Table

In [0]:
factTrip_df = spark.sql('''SELECT  DISTINCT trip.trip_id as trip_key,
                    trip.trip_id as trip_id,
                    td1.date_key as start_date_key,  
                    td2.date_key as end_date_key,               
                    rider.rider_key as rider_key,
                    DATEDIFF(second, trip.start_at,trip.ended_at) as duration,
                    st1.station_key as start_station_key,
                    st2.station_key as end_station_key,
                    DATEDIFF(MONTH, rider.birthday,trip.start_at) as rider_age,                    
                    trip.rideable_type as rideable_type
            FROM bronze_trips trip
            LEFT JOIN silver_dimDate td1 ON date_trunc("Day",td1.datetime) = date_trunc("Day",trip.start_at)
            LEFT JOIN silver_dimDate td2 ON DATE_TRUNC("Day",td2.datetime) = DATE_TRUNC("Day",trip.ended_at)
            JOIN silver_dimRider rider ON trip.rider_id = rider.rider_id
            JOIN silver_dimStation st1 ON trip.start_station_id = st1.station_id
            JOIN silver_dimStation st2 ON trip.end_station_id = st2.station_id''')
factTrip_df.write.format("delta").mode("overwrite").saveAsTable("silver_factTrip")

## Business Queries for which Gold Layer tables are created.

### Analyze how much time is spent per ride Based on time of day
### gold_duration_by_hour

In [0]:
dur_by_hour_df = spark.sql('''SELECT AVG(DURATION) , dt.HOUR
FROM silver_factTrip trip
INNER JOIN silver_dimDate dt
ON trip.start_date_key = dt.start_date_key
group by dt.hour''')
dur_by_hour_df.write.format("delta").mode("overwrite").saveAsTable("gold_duration_by_hour")

### Analyze how much time is spent per ride Based on which station is the starting and / or ending station
### gold_duration_by_station

In [0]:
dur_by_station_df = spark.sql('''SELECT AVG(trip.duration),st.name
FROM silver_factTrip trip
INNER JOIN silver_dimStation st
ON trip.start_station_key = st.station_key
WHERE ST.NAME = 'Glenwood Ave & Touhy Ave'
GROUP BY st.name''')
dur_by_station_df.write.format("delta").mode("overwrite").saveAsTable("gold_duration_by_station")

### Analyze how much time is spent per ride Based on age of the rider at time of the ride
### gold_duration_by_riderage

In [0]:
dur_by_age_df = spark.sql('''
SELECT AVG(duration), age
FROM silver_factTrip
group by age''')
dur_by_age_df.write.format("delta").mode("overwrite").saveAsTable("gold_duration_by_riderage")

### Analyze how much money is spent month
### gold_amount_by_month

In [0]:
amt_by_mnt_df = spark.sql('''SELECT SUM(amount) , dt.month, dt.year, dt.quarter
FROM silver_factPayment p
INNER JOIN silver dimDate dt
ON p.date_key = dt.date_key
where dt.year = '2021'
GROUP BY dt.month, dt.year, dt.quarter''')
amt_by_mnt_df.write.format("delta").mode("overwrite").saveAsTable("gold_amount_by_month")

### Analyze how much money is spent Per member, based on the age of the rider at account start
### gold_amount_by_rider

In [0]:
amount_by_rider_df = spark.sql('''SELECT SUM(AMOUNT), R.FIRST_NAME
FROM silver_factPayment p
INNER JOIN silver_dimRider r
WHERE r.age = 25
GROUP BY r.first_name''')
amount_by_rider_df.write.format("delta").mode("overwrite").saveAsTable("gold_amount_by_rider")

### Analyze how much money is spent per member Based on how many rides the rider averages per month
### gold_rides_per_rider_month

In [0]:
rides_df = spark.sql('''SELECT count(trip_id), r.first_name,r.rider_key, dt.month
FROM silver_factTrip t
INNER JOIN silver_dimRider r ON t.rider_key = r.rider_key
INNER JOIN silver_dimDate d ON t.date_key = d.date_key
GROUP BY r.first_name , r.rider_key,dt.month''')
rides_df.write.format("delta").mode("overwrite").saveAsTable("gold_rides_per_rider_month")



### Analyze how much money is spent per member Based on how many minutes the rider spends on a bike per month
### gold_duration_rider

In [0]:

dur_rider_df = spark.sql('''SELECT sum(minutes) , r.rider_key, r.first_name
FROM silver_factTrip t
INNER JOIN silver_dimDate d ON t.date_key = d.date_key
INNER JOIN silver_dimRider r on t.rider_key = r.rider_key''')
dur_rider_df.write.format("delta").mode("overwrite").saveAsTable("gold_duration_rider")